# LIBRERIAS GENERALES

In [ ]:
import pandas as pd
import statistics
from sklearn.metrics.pairwise import cosine_similarity
from Funciones.json_func import *
from Funciones.textblob import *
from Funciones.dates import *

.

.

.

.

# TRATAMIENTO DE STEAM GAMES

Establecemos las columnas relevantes para el MVP

In [ ]:
cols_output_steam_json = ['item_id', 'genres', 'release_date', 'price', 'developer'] 

Lectura del Json.Gz de STEAM GAMES

In [ ]:
#Previo a las Creacion de Particiones
#df_output_steam_json_entero = leerJsonGz("Datos/","steam_games.json.gz",0)

In [ ]:
#Lectura y union de particiones
df_output_steam_json = leerJsonGzPart("Datos/Particiones/","steam_games.json.gz",0,3)

In [ ]:
#Se comprobo que funciono el particionamiento de este archivo
#print("Shape lectura completa    : ", df_output_steam_json_entero.shape)
#print("Shape lectura particionada: ", df_output_steam_json.shape)


In [ ]:
df_output_steam_json.head()

In [ ]:
df_output_steam_json.shape

In [ ]:
# Renombra la columna "id" a "item_id"
df_output_steam_json.rename(columns={"id": "item_id"}, inplace=True)
df_output_steam_json.head()

In [ ]:
df_output_steam_json[df_output_steam_json['item_id'] == '10']

VALIDAMOS LA ESTRUCTURA ITEMS (anidada) DEL ARCHIVO STEAM GAMES

Se detecto que si viene informado ITEMs el resto de los datos es nulo.

Ademas, esa informacion ya la tenemos en USER ITEMs. Asi que sera descartada por redundancia de datos.

In [ ]:
df_output_steam_json[df_output_steam_json["items"].notna()][['item_id', 'genres', 'release_date', 'price', 'developer', 'items']].info()

In [ ]:
cols_output_steam_json

In [ ]:
#Nos quedamos con las columnas necesarias para los EndPoints
df_output_steam_json = df_output_steam_json[cols_output_steam_json]

In [ ]:
df_output_steam_json[cols_output_steam_json].head()

ELIMINAMOS REGISTROS SIN INFORMACION, GENERADOS POR LA ESTRUCTURA ITEMs

In [ ]:
df_output_steam_json.shape[0]

In [ ]:
#Solo elimina las filas donde todos sus campos o columnas sean Nulas.
df_output_steam_json = df_output_steam_json.dropna(how='all')

In [ ]:
df_output_steam_json.shape[0]

In [ ]:
df_output_steam_json.head()

OBSERVAMOS LA PRESENCIA DE NULOS EN LAS COLUMNAS SELECCIONADAS

In [ ]:
df_output_steam_json.info()

.

.

TRATAMIENTO DE NULOS CAMPO A CAMPO

.

.

ITEM-ID

In [ ]:
#observamos la cantidad de registros nulos de ITEM-ID
df_output_steam_json[df_output_steam_json["item_id"].isna()].shape[0]

In [ ]:
#observamos los registros nulos de ITEM-ID
df_output_steam_json[df_output_steam_json["item_id"].isna()]

Procedemos a la eliminacion de los registros sin Item_Id ya que no hay forma de imputarlo o reconstruirlo con los datos que tenemos

In [ ]:
#Cantidad de Registros previo a la eliminacion
df_output_steam_json.shape[0]

In [ ]:
# Elimina las filas donde "item_id" sea nulo
df_output_steam_json = df_output_steam_json.dropna(subset=["item_id"])

In [ ]:
#Cantidad de Registros luego de la eliminacion
df_output_steam_json.shape[0]

In [ ]:
#Convertimos la columna a Entero
df_output_steam_json['item_id'] = df_output_steam_json['item_id'].astype(int)

GENRES

VALIDO CUANTOS GENROS DIFERENTES TENGO PREVIO A CUALQUIER MODIFICACION

In [ ]:
# Convierto en string
# Si no es lista se pasa directo a string
#Si es lista, se conveierte a cadena, separando los elementos (generos) por coma.
genres_str = df_output_steam_json["genres"].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))

# Dividir la columna 'genres' en géneros individuales y crear un conjunto de géneros únicos
unique_genres_set = set(genres_str.str.split(',').explode().str.strip())

# Convertir el conjunto de géneros únicos en una lista
unique_genres_list = list(unique_genres_set)

# Mostrar la lista de géneros únicos
print(unique_genres_list)
print(len(unique_genres_list))


In [ ]:
#observamos la cantidad de registros nulos en el campo GENRES
df_output_steam_json[df_output_steam_json["genres"].isna()].shape[0]

In [ ]:
df_output_steam_json[df_output_steam_json["genres"].isna()].head()

Debido a que es muy dificil obtener un resultado CONFIABLE al tratar de inferir los generos con el resto de la informacion disponible, se ha optado por hacer un ETIQUETADO especial a esos registros con generos nulos.

Se procedera a imputar ["Desconocido] a todos esos casos con el objetivo de no perder informacion relevante como ser: precios, fechas, etc.

In [ ]:
#eliminamos los elementos o generos 'Free To Play' y 'Early Access' ya que condiseramos no ser un genero.
df_output_steam_json['genres'] = df_output_steam_json['genres'].apply(lambda x: [genre for genre in x if isinstance(x, list) and genre != 'Free to Play'] if isinstance(x, list) else x)
df_output_steam_json['genres'] = df_output_steam_json['genres'].apply(lambda x: [genre for genre in x if isinstance(x, list) and genre != 'Early Access'] if isinstance(x, list) else x)

#Si solamente tenia ese genero, me va a quedar una lista vacia, en esos casos imputo Desconocido
df_output_steam_json['genres'] = df_output_steam_json['genres'].apply(lambda x: ['Desconocido'] if (isinstance(x, list) and (len(x) == 0 or (len(x) == 1 and x[0] == ''))) else x)

In [ ]:
# Convierto en string
# Si no es lista se pasa directo a string
#Si es lista, se conveierte a cadena, separando los elementos (generos) por coma.
genres_str = df_output_steam_json["genres"].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))

# Dividir la columna 'genres' en géneros individuales y crear un conjunto de géneros únicos
unique_genres_set = set(genres_str.str.split(',').explode().str.strip())

# Convertir el conjunto de géneros únicos en una lista
unique_genres_list = list(unique_genres_set)

# Mostrar la lista de géneros únicos
print(unique_genres_list)
print(len(unique_genres_list))

transformo la columna de genres en string

In [ ]:
# Verifica el contenido de GENRES
# Si no es lista se pasa directo a string
#Si es lista, se conveierte a cadena, separando los elementos (generos) por coma.
df_output_steam_json["genres"] = df_output_steam_json["genres"].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x)).str.strip()

In [ ]:
# Verifica y asigna "Desconocido" a registros nulos y vacios en la columna "genres"
df_output_steam_json['genres'] = df_output_steam_json['genres'].apply(lambda x: "Desconocido" if x == 'nan' else x)
df_output_steam_json['genres'] = df_output_steam_json['genres'].apply(lambda x: "Desconocido" if x == '' else x)

In [ ]:
#observamos los registros nulos tras la imputacion
df_output_steam_json[df_output_steam_json["genres"].isna()].shape[0]

In [ ]:
df_output_steam_json[df_output_steam_json['genres'] == ''].shape[0]

In [ ]:
# Sustituir la subcadena 'Design &amp; ' por ''
df_output_steam_json['genres'] = df_output_steam_json['genres'].str.replace('Design &amp; ', '')
df_output_steam_json['genres'] = df_output_steam_json['genres'].str.replace('Animation &amp; ', '')
df_output_steam_json['genres'] = df_output_steam_json['genres'].str.rstrip(', ')

In [ ]:
df_output_steam_json[df_output_steam_json['genres'] == ''].shape[0]

In [ ]:
df_output_steam_json.head()

RELEASE_DATE

In [ ]:
#observamos los registros nulos
df_output_steam_json[df_output_steam_json["release_date"].isna()].shape[0]

Debido a que es muy dificil obtener un resultado CONFIABLE al tratar de inferir las FECHAS de LANZAMIENTO con el resto de la informacion disponible, se ha optado por imputar un valor especial a esos registros con fechas nulas.

Se procedera a imputar 1900-01-01 a todos esos casos con el objetivo de no perder informacion relevante.

In [ ]:
#Previo a dicha imputacion comprobamos que esa fecha NO se este utilizando
df_output_steam_json[df_output_steam_json["release_date"].str.strip() == "1900-01-01"].shape[0]

In [ ]:
#Visualizo un registro para ver como imputa la fecha default
df_output_steam_json[df_output_steam_json["release_date"].isna()].head(1)

In [ ]:
# Asigna '1900-01-01' a los registros nulos en la columna "release_date"
df_output_steam_json["release_date"].fillna(pd.to_datetime(DEFAULT_DATE).date().strftime("%Y-%m-%d"), inplace=True)

In [ ]:
#observamos los registros nulos tras la imputacion
df_output_steam_json[df_output_steam_json["release_date"].isna()].shape[0]

In [ ]:
#Visualizo el registro anterior post imputacion de DEFAULT DATE
df_output_steam_json[df_output_steam_json["item_id"] == 773570]

Necesitamos verificar la diversidad de posibilidades que nos podemos encontrar ya que manualmente hemos observado muchos tipos de fechas. 

Vamos a Realizar una separacion de los datos que no estan bajo el formato YYYY-MM-DD

In [ ]:
df_output_steam_json.head()

In [ ]:
#Casteamos a String
df_output_steam_json['release_date'] = df_output_steam_json['release_date'].astype(str)

# Aplica la función de conversión a la columna 'release_date'
df_output_steam_json['converted_release_date'] = df_output_steam_json['release_date'].apply(convertReleaseFuzzy)

VALIDAMOS QUE TODAS LAS FECHAS SE HAYAN TRATADO CORRECTAMENTE

In [ ]:
# Filtra los registros que no se pudieron convertir
failed_conversion_df = df_output_steam_json[df_output_steam_json['converted_release_date'].isna()]

# Muestra el nuevo DataFrame con el conteo de valores
print("Cantidad de Fechas que no pudieron ser tratadas: ", failed_conversion_df.shape[0])

In [ ]:
df_output_steam_json[df_output_steam_json['release_date'] == pd.to_datetime(DEFAULT_DATE).date().strftime("%Y-%m-%d")]

In [ ]:
df_output_steam_json[(df_output_steam_json['converted_release_date'] == DEFAULT_DATE) & (df_output_steam_json['release_date'] != DEFAULT_DATE)]

In [ ]:
df_output_steam_json.head()

In [ ]:
#Asignamos una columna a la otra
df_output_steam_json['release_date'] = df_output_steam_json['converted_release_date']
#eliminamos Columna intermedia 
df_output_steam_json.drop(columns=['converted_release_date'], inplace=True)

GENERAMOS COLUMNA YEAR que nos sera util para los endpoints

In [ ]:
# Extraer el año de la columna 'release_date'
df_output_steam_json['year'] = df_output_steam_json['release_date'].str[:4]

# Convertir la columna 'year' a tipo numérico
df_output_steam_json['year'] = pd.to_numeric(df_output_steam_json['year'], errors='coerce')

In [ ]:
#convertimos a String
df_output_steam_json['release_date'] = df_output_steam_json['release_date'].astype(str)

In [ ]:
df_output_steam_json.head()

PRICE

In [ ]:
# Convierte la columna 'price' a numérica, y los valores no válidos se convierten en NaN
df_output_steam_json['price_convertidos'] = pd.to_numeric(df_output_steam_json['price'], errors='coerce')

In [ ]:
df_output_steam_json[df_output_steam_json['item_id'] == 761140]

In [ ]:
#vemos que price tenian aquellos que no hemos podido convertir para ver como tratarlos
df_output_steam_json[df_output_steam_json['price_convertidos'].isna()].head()

In [ ]:
#nos quedamos con los valores unicos de esos Prices extraños
df_output_steam_json[df_output_steam_json['price_convertidos'].isna()]['price'].unique()

Realizamos las transformaciones de esos casos extraños

In [ ]:
#Casos con Starting at $
df_output_steam_json['price'] = df_output_steam_json['price'].apply(lambda x: x.replace('Starting at $', '') if isinstance(x, str) and 'Starting at $' in x else x)

In [ ]:
#Modificamos los FREE
# Utiliza str.contains() para identificar registros que contienen "free" (sin importar mayúsculas o minúsculas)
# Modifica los registros que cumplen con la condición a 0.0
df_output_steam_json.loc[df_output_steam_json['price'].str.contains('free', case=False, na=False), 'price'] = 0.0

Volvemos a aplicar la conversion...

In [ ]:
# Convierte la columna 'price' a numérica, y los valores no válidos se convierten en NaN
df_output_steam_json['price_convertidos'] = pd.to_numeric(df_output_steam_json['price'], errors='coerce')

In [ ]:
#nos quedamos con los valores unicos de esos Prices extraños
df_output_steam_json[df_output_steam_json['price_convertidos'].isna()]['price'].unique()

ASUMIMOS COMO AUSENCIA DE INFORMACION PARA ESOS CASOS, CON LO CUAL QUEDARAN CON NaN luego de la transformacion.

In [ ]:
df_output_steam_json['price'] = df_output_steam_json['price_convertidos']
df_output_steam_json.drop('price_convertidos', axis=1, inplace=True)

In [ ]:
df_output_steam_json.head()

In [ ]:
df_output_steam_json[df_output_steam_json['item_id'] == 761140]

DEVELOPER

In [ ]:
#Visualizamos algunos registros con DEVELOPER nulo
df_output_steam_json[df_output_steam_json['developer'].isna()]

In [ ]:
#Cantidad total de nulos en columna DEVELOPER previo a la imputacion
df_output_steam_json[df_output_steam_json['developer'].isna()].shape[0]

In [ ]:
# Verifica y asigna "Desconocido" a registros nulos en la columna "developer"
df_output_steam_json['developer'] = df_output_steam_json['developer'].apply(lambda x: "Desconocido" if pd.isna(x) else x)

In [ ]:
#Cantidad total de nulos en columna DEVELOPER luego de la imputacion
df_output_steam_json[df_output_steam_json['developer'].isna()].shape[0]

In [ ]:
#Visualizamos algunos registros
df_output_steam_json.head()

In [ ]:
#Cantidad de Developers diferentes
df_output_steam_json['developer'].unique().shape[0]

In [ ]:
df_output_steam_json['developer'] = df_output_steam_json['developer'].str.strip()

In [ ]:
#Cantidad de Developers diferentes
df_output_steam_json['developer'].unique().shape[0]

UNA VEZ FINALIZADAS TODAS LAS TRANSFORMACIONES VOLVEMOS A ELIMINAR DUPLICADOS POR LAS DUDAS

In [ ]:
#Vemos cantidad de Registros antes de la eliminacion
df_output_steam_json.shape[0]

In [ ]:
# Elimina registros duplicados en el DataFrame
df_output_steam_json.drop_duplicates(inplace=True)

In [ ]:
#Vemos cantidad de Registros despues de la eliminacion
df_output_steam_json.shape[0]

In [ ]:
#Reseteamos indices
df_output_steam_json.reset_index(drop=True, inplace=True)

In [ ]:
#Convertimos a string
df_output_steam_json['developer'] = df_output_steam_json['developer'].astype(str)

.

.

.

# USER REVIEWS

Establecemos las columnas relevantes para el MVP

In [ ]:
cols_user_reviews = ["user_id", "posted", "item_id", "recommend", "review"]

LECTURA

In [ ]:
#Lectura previa al particionamiento
#df_user_reviews_entero = leerJsonGz("Datos/", "user_reviews.json.gz",1)

In [ ]:
#Lectura y union de particiones
df_user_reviews = leerJsonGzPart("Datos/Particiones/","user_reviews.json.gz",0,3)

In [ ]:
#Se comprobo que el particionamiento ha funcionado
#print("Shape Lectura Completa    : ",df_user_reviews_entero.shape)
#print("Shape Lectura Particionada: ",df_user_reviews.shape)

In [ ]:
#visualizamos DF
df_user_reviews.head()

DESANIDAMOS

Invoca a la funcion que desanida los json. En este caso, abrira en tantas columnas como claves tengan los diccionarios que estan dentro de la estructura REVIEWS

In [ ]:
print("Cantidad de Registros sin desanidar: ", df_user_reviews.shape[0])
print("Cantidad de Columnas  sin desanidar: ", df_user_reviews.shape[1])

In [ ]:
df_user_reviews = desanidarJson(df_user_reviews, 'reviews')
df_user_reviews.head()

In [ ]:
print("Cantidad de Registros luego de desanidar: ", df_user_reviews.shape[0])
print("Cantidad de Columnas  luego de desanidar: ", df_user_reviews.shape[1])

In [ ]:
#Me quedo con las columnas Relevantes para el MVP
df_user_reviews = df_user_reviews[cols_user_reviews]
df_user_reviews.head()

ELIMINO REGISTROS QUE NO ME SIRVEN

- Cuando review y recommend estan sin informacion, ya que no me aportan ninguna informacion esos registros.

In [ ]:
df_user_reviews = df_user_reviews.dropna(subset=['recommend', 'review'], how='all')
df_user_reviews.reset_index(drop=True, inplace=True)
df_user_reviews.shape

In [ ]:
df_user_reviews.info()

==> Podemos ver que no hay mas valores nulos

.



TRANSFORMACIONES

.



POSTED

La fecha Viene sin año en algunos casos y ademas esta en un formato poco util. vamos a proceder a llevarla a YYYY-MM-DD al igual que lo hicimos con RELEASE_DATE.

In [ ]:
#Quito Espacios
df_user_reviews['posted'] = df_user_reviews['posted'].str.strip()
#Quitamos palabra 'Posted' y el '.' final.
df_user_reviews['posted'] = df_user_reviews['posted'].str.replace('Posted ', '').str.replace('.', '')
#No hay nulos, pero si los hubiera completariamos con una fecha default
df_user_reviews['posted'].fillna(DEFAULT_DATE_LONG, inplace=True)
# Realiza el cambio de formato de la FECHA hacia YYYY-MM-DD
df_user_reviews['posted_new'] = df_user_reviews['posted'].apply(convertPosted)
df_user_reviews.head()

Confirmamos nuevamente que no hay posteos sin fecha

In [ ]:
print("Cantidad de Fechas Default en USER REVIEWs: ", df_user_reviews[df_user_reviews["posted"] == "1900-01-01"].shape[0])

In [ ]:
df_user_reviews["posted"].isna().value_counts()

Veo cuantas fechas hay con año 1900

In [ ]:
df_user_reviews[df_user_reviews['posted_new'].str.contains('1900')]

Se pudo ver que las fechas, luego de aplicar el drop de registros sin recommend y sin revew, vienen siempre informadas. Pero hay un defecto. Hay fechas sin año, es decir, viene solo mes y dia.

==> Imputamos como año de Posteo el correspondiente a La fecha de Lanzamiento del Contenido:

In [ ]:
# Creamos series booleanas para buscar las fechas con año 1900 pero que no tengan dia y mes 01.
#Si tiene año 1900 y no es 01 el dia y mes, quiere decir que es una feha que vino informada SIN año. En ese caso imputamos año de lantamiento si es que lo tenemos. Sino DEFAULT_DATE.
es_1900  = df_user_reviews['posted_new'].str[:4] == "1900"
no_es_01 = df_user_reviews['posted_new'].str[4:9] != "01-01"
# Combina las condiciones
es_1900_y_no_es_01 = es_1900 & no_es_01
# Filtra el DataFrame para obtener las filas que cumplen con las condiciones
df_user_reviews[es_1900_y_no_es_01].head(10)

In [ ]:
#comprobacion de un caso puntual con DEFAULT
df_user_reviews[(df_user_reviews['item_id'] == '248820') & (df_user_reviews['user_id'] == 'evcentric')]

In [ ]:
#Vemos cuantos casos de estos tenemos en nuestro data frame
df_user_reviews[es_1900_y_no_es_01].shape[0]

In [ ]:
#Reseteamos indices
df_user_reviews.reset_index(drop=True, inplace=True)

In [ ]:
df_user_reviews_filtrado = df_user_reviews[es_1900_y_no_es_01]

# Para cada registro en los registros filtrados
for indice, fila in df_user_reviews_filtrado.iterrows():
    item_id = fila['item_id']  # Obtiene el valor de 'item_id' de df_user_reviews
    
    # Busca las coincidencias para item_id en df_output_steam_json
    coincidencias = df_output_steam_json[df_output_steam_json['item_id'] == item_id]
    
    # Verifica si se encontraron coincidencias antes de intentar acceder al valor
    if not coincidencias.empty:
        # Obtén el primer valor de 'release_date' en las coincidencias
        anio_release = coincidencias['release_date'].str[:4].values[0]
        # Actualiza solo los primeros 4 caracteres de 'posted_new' en df_user_reviews
        df_user_reviews.at[indice, 'posted_new'] = anio_release + df_user_reviews.at[indice, 'posted_new'][4:]

In [ ]:
df_user_reviews[es_1900_y_no_es_01].head(10)

In [ ]:
#comprobacion de un caso puntual con DEFAULT
df_output_steam_json[df_output_steam_json['item_id'] == 218620]

Ahora solo aplicamos DEFAULT_DATE para aquellas fechas de posteo que aun tengan año 1900

In [ ]:
# Filtra el DataFrame para obtener las filas que cumplen con las condiciones
es_1900  = df_user_reviews['posted_new'].str[:4] == "1900"
df_user_reviews[es_1900].head(10)

In [ ]:
#validacion de caso puntual item_id = 271290
#Como no se encontro en el df, no actualizo el año
df_output_steam_json[df_output_steam_json['item_id'] == 271290]

In [ ]:
#imputamos DEFAULT_DATE en estos casos

# Filtro para identificar los registros que cumplen con es_1900
registros_es_1900 = df_user_reviews[es_1900]

# Modificar la columna 'posted_new' en los registros filtrados
registros_es_1900.loc[:, 'posted_new'] = DEFAULT_DATE

# Actualizar df_user_reviews con los cambios
df_user_reviews.update(registros_es_1900)

In [ ]:
df_user_reviews[es_1900].head(10)

In [ ]:
df_user_reviews.head(10)

In [ ]:
#Reemplazamos la columna de posted y eliminamos la intermedia
df_user_reviews['posted'] = df_user_reviews['posted_new']
df_user_reviews.drop('posted_new', axis=1, inplace=True)

GENERAMOS COLUMNA YEAR

In [ ]:
# Extrae el año de la columna 'posted' en df_user_reviews. Esta columna sera util para los endpoints
df_user_reviews['year'] = df_user_reviews['posted'].str[:4]
# Convertir la columna 'year' a tipo numérico
df_user_reviews['year'] = pd.to_numeric(df_user_reviews['year'], errors='coerce')

.



RECOMMEND

In [ ]:
df_user_reviews['recommend'].value_counts()

In [ ]:
# Cambiar los valores de True a 1 y False a 0 en la columna 'recommend'
df_user_reviews['recommend'] = df_user_reviews['recommend'].astype(int)
df_user_reviews['recommend'].value_counts()

In [ ]:
df_user_reviews.head(10)

In [ ]:
#Volvemos a confirmar que no tenemos NULOS
df_user_reviews.info()

ITEM-ID

In [ ]:
#Convertimos a int
df_user_reviews['item_id'] = df_user_reviews['item_id'].astype(int)

.

.

.

# ANALISIS DE SENTIMIENTO

Libreria Usada: Text Blob

In [ ]:
df_user_reviews = pd.concat([df_user_reviews, sentimientoTextblob(df_user_reviews["review"], "sentiment_analysis")], axis = 1)
df_user_reviews.head()

In [ ]:
print("Cantidad de Reviews Negativas: ",df_user_reviews[df_user_reviews["sentiment_analysis"] == 0].shape[0])
print("Cantidad de Reviews Neutras  : ",df_user_reviews[df_user_reviews["sentiment_analysis"] == 1].shape[0])
print("Cantidad de Reviews Positivas: ",df_user_reviews[df_user_reviews["sentiment_analysis"] == 2].shape[0])
print("Cantidad TOTAL de Reviews    : ",df_user_reviews.shape[0])

In [ ]:
df_user_reviews.shape[0]

In [ ]:
#Convertimos a int
df_user_reviews['sentiment_analysis'] = df_user_reviews['sentiment_analysis'].astype(int)

.

.

.

.

.

.
 
.

# TRATAMIENTO DE USER ITEMS

SELECCIONAMOS COLUMNAS RELEVANTES PARA EL MVP

In [ ]:
cols_user_items = ['user_id', 'user_url', 'playtime_forever', 'item_id'] 

LECTURA

In [ ]:
#Lectura previa al particionamiento
#df_user_items_entero = leerJsonGz("Datos/","users_items.json.gz",1)

In [ ]:
#Lectura y union de particiones
df_user_items = leerJsonGzPart("Datos/Particiones/","users_items.json.gz",1,3)

In [ ]:
#Se comprobo que el particionamiento ha funcionado correctamente
#print("Shape Lectura Completa    : ", df_user_items_entero.shape)
#print("Shape Lectura Particionada: ", df_user_items.shape)

In [ ]:
df_user_items.head()

DESANIDAMOS

In [ ]:
df_user_items = desanidarJson(df_user_items, "items")
df_user_items.head()

ELIMINAMOS DUPLICADOS

In [ ]:
df_user_items.shape[0]

In [ ]:
#Vemos cantidad de Registros
df_user_items.shape[0]

In [ ]:
df_user_items.drop_duplicates(inplace=True)

In [ ]:
#Vemos cantidad de Registros
df_user_items.shape[0]

PLAYTIME 2 WEEKS y PLAYTIME FOREVER

In [ ]:
#Conversion a Numerico
df_user_items["playtime_2weeks"]  = pd.to_numeric(df_user_items["playtime_2weeks"], errors='coerce')
df_user_items["playtime_forever"] = pd.to_numeric(df_user_items["playtime_forever"], errors='coerce')

In [ ]:
#Vemos Nulos en playtime_2weeks
df_user_items[df_user_items["playtime_2weeks"].isna()].shape[0]

In [ ]:
#Vemos Nulos en playtime_forever
df_user_items[df_user_items["playtime_forever"].isna()].shape[0]

In [ ]:
#Vemos si los nulos se dan en ambas al mismo tiempo
df_user_items[(df_user_items["playtime_2weeks"].isna()) & (df_user_items["playtime_forever"].isna())].shape[0]

Si ambos estan en nulo, los pongo ambos en cero

In [ ]:
#Definimos la condicion
ambos_nulos = (df_user_items["playtime_2weeks"].isna()) & (df_user_items["playtime_forever"].isna())
# Actualizar los valores en las columnas 'playtime_2weeks' y 'playtime_forever' a 0.0
df_user_items.loc[ambos_nulos, ['playtime_2weeks', 'playtime_forever']] = 0.0

#Verificamos
df_user_items[(df_user_items["playtime_2weeks"].isna()) & (df_user_items["playtime_forever"].isna())].shape[0]

PLAYTIME FOREVER = 0 & PLAYTIME 2 WEEKS != 0

In [ ]:
#Reseteo indices
df_user_items.reset_index(drop=True, inplace=True)

In [ ]:
#Primero cambiamos los tipos de datos de ambas
df_user_items["playtime_forever"] = df_user_items["playtime_forever"].astype(float)
df_user_items["playtime_2weeks"]  = df_user_items["playtime_2weeks"].astype(float)

In [ ]:
#Creamos la condicion/mascara
condicion = ((df_user_items["playtime_forever"] == 0) & (df_user_items["playtime_2weeks"] != 0))

#Visualizamos algunos registros
df_user_items[condicion].head()

In [ ]:
#Cantida de registros con PLAYTIME FOREVER = 0 & PLAYTIME 2 WEEKS != 0
df_user_items[condicion].shape[0]

In [ ]:
#Asigno el valor de playtime_2weeks a playtime_forever
# Asignar el valor de 'playtime_2weeks' a 'playtime_forever' en los registros que cumplen la condición
df_user_items.loc[condicion, 'playtime_forever'] = df_user_items.loc[condicion, 'playtime_2weeks']

In [ ]:
#Cantida de registros con PLAYTIME FOREVER = 0 & PLAYTIME 2 WEEKS != 0 POST imputaicon
df_user_items.iloc[253243]

OTRAS TRANSFORMACIONES NECESARIAS

In [ ]:
#Imputamos -1 en item ID para los casos nulos
df_user_items['item_id'].fillna(-1, inplace=True)
#Convertimos a Int
df_user_items['item_id'] = df_user_items['item_id'].astype(int)

.

.

.

.

# GENERACION DE ARCHIVOS REDUCIDOS PARA C/ ENDPOINT

.

.

1) **<u> def userdata( User_id : str ):</u>**

    Debe devolver cantidad de dinero gastado por el usuario, 
    el porcentaje de recomendación en base a reviews.recommend y 
    cantidad de items.

CREAREMOS 3 DFs REDUCIDOS (COMO SI FUESEN  VISTAS) para cada solicitud del ENDPOINT

df_price_by_user

df_perc_recommend_by_user

df_cant_items_by_user

In [ ]:
#Usuarios Unicos
df_user_items["user_id"].unique().shape[0]

.

**<u>df_price_by_user:</u>**   Va a ser un DF agrupado por user_id con la sumatoria de todos los gastos (price) efectuados por ese usuario.

.

In [ ]:
#Creamos DF vacio
df_spent_by_user = pd.DataFrame()

#Agrego Columna de user_id
df_spent_by_user["user_id"] = df_user_items["user_id"].unique()

#verifico cantidad de registros
df_spent_by_user.shape[0]

In [ ]:
#Creamos un DataFrame entre steam y users para tener el precio a nivel item_id
df_merged_items_steam_by_itemid = df_user_items.merge(df_output_steam_json, on='item_id', how='left')
#Por las dudas convertimos a float la columna precio
df_merged_items_steam_by_itemid['price'] = df_merged_items_steam_by_itemid['price'].astype(float)

df_merged_items_steam_by_itemid.head()

In [ ]:
#Total de Gastos/price (sum) por usuario
df_spent_by_user = pd.merge(df_spent_by_user, df_merged_items_steam_by_itemid.groupby('user_id')['price'].sum().reset_index(), on='user_id', how='left')
#Modificamos el nombre de la columna calculada
df_spent_by_user.rename(columns={"price": "spent"}, inplace=True)
#Visualizamos el df
df_spent_by_user.head()

.



**<u>df_perc_recommend_by_user:</u>**   Va a ser un DF agrupado por user_id el porcentaje de recomendaciones POSITIVAS(1) en funcion de las NEGATIVAS(0).

.



In [ ]:
#Creamos un DF vacio
df_perc_recommend_by_user = pd.DataFrame()

#Agrego Columna de user_id
df_perc_recommend_by_user["user_id"] = df_user_items["user_id"].unique()

#verifico cantidad de registros
df_perc_recommend_by_user.shape[0]

In [ ]:
#Recomendaciones Negativas
df_recommend_0 = df_user_reviews[df_user_reviews['recommend'] == 0].groupby('user_id')['recommend'].count().reset_index()
df_recommend_0.rename(columns={"recommend": "recommend_0"}, inplace=True)
#Recomendaciones positivas
df_recommend_1 = df_user_reviews[df_user_reviews['recommend'] == 1].groupby('user_id')['recommend'].count().reset_index()
df_recommend_1.rename(columns={"recommend": "recommend_1"}, inplace=True)

#Combinamos ambos DFs
df_recommend = pd.merge(df_recommend_0,df_recommend_1, on='user_id', how='inner')

#Actuaslizamos los valores nulos
df_recommend['recommend_0'].fillna(0, inplace=True)
df_recommend['recommend_1'].fillna(0, inplace=True)

# Convierte las columnas 'recommend_0' y 'recommend_1' a enteros
df_recommend['recommend_0'] = df_recommend['recommend_0'].astype(int)
df_recommend['recommend_1'] = df_recommend['recommend_1'].astype(int)


#Calculamos el total de recomendaciones por usuario en una nueva columna, sumando las positivas y negativas
df_recommend['recommend_total'] = df_recommend['recommend_0'] + df_recommend['recommend_1']
df_recommend['recommend_total'] = df_recommend['recommend_total'].astype(int)


#Calculamos el porcentaje con las columnas anteriores
df_recommend['perc_recommend'] = 0  # Inicializa la columna con ceros
df_recommend['perc_recommend'] = df_recommend['perc_recommend'].astype(float)

nonzero_total_mask = df_recommend['recommend_total'] != 0 #Crea Mascara, porque sino podriamos generar una division por cero. 
df_recommend.loc[nonzero_total_mask, 'perc_recommend'] = (df_recommend['recommend_1'] / df_recommend['recommend_total']) * 100

#Quitamos intermedias
#df_recommend.drop(['recommend_0', 'recommend_1', 'recommend_total'], axis=1, inplace=True)


In [ ]:
df_recommend.info()

In [ ]:
df_recommend.shape

In [ ]:
df_recommend.head()

In [ ]:
#Juntamos con nuestro "maestro" de user_id el df anterior
df_perc_recommend_by_user = pd.merge(df_perc_recommend_by_user,df_recommend, on='user_id', how='left')

# Reemplaza los valores nulos por ceros 
df_perc_recommend_by_user['recommend_0'].fillna(0, inplace=True)
df_perc_recommend_by_user['recommend_1'].fillna(0, inplace=True)
df_perc_recommend_by_user['recommend_total'].fillna(0, inplace=True)
df_perc_recommend_by_user['perc_recommend'].fillna(0, inplace=True)

# Convierte el tipo de dato de la columna 'perc_recommend' a float
df_perc_recommend_by_user['recommend_0'] = df_perc_recommend_by_user['recommend_0'].astype(float)
df_perc_recommend_by_user['recommend_1'] = df_perc_recommend_by_user['recommend_1'].astype(float)
df_perc_recommend_by_user['recommend_total'] = df_perc_recommend_by_user['recommend_total'].astype(float)
df_perc_recommend_by_user['perc_recommend'] = df_perc_recommend_by_user['perc_recommend'].astype(float)

In [ ]:
df_perc_recommend_by_user.head()

.


**<u>df_cant_items_by_user:</u>**   Va a ser un DF agrupado por user_id Con la cantidad de items de cada user.

.

In [ ]:
#Creamos un DF vacio
df_count_items_by_user = pd.DataFrame()

#Agrego Columna de user_id
df_count_items_by_user["user_id"] = df_user_items["user_id"].unique()

#verifico cantidad de registros
df_count_items_by_user.shape[0]

In [ ]:
#Agregamos la columna de cantidad de items
df_count_items_by_user = pd.merge(df_count_items_by_user, df_user_items.groupby('user_id')['item_id'].count().reset_index(), on='user_id', how='left')
#Modificamos el nombre de la columna calculada
df_count_items_by_user.rename(columns={"item_id": "count_items"}, inplace=True)
# Reemplaza los valores nulos por ceros 
df_count_items_by_user['count_items'].fillna(0, inplace=True)
# Convierte el tipo de dato de la columna 'perc_recommend' a float
df_count_items_by_user['count_items'] = df_count_items_by_user['count_items'].astype(int)
#Visualizamos el DF
df_count_items_by_user.head()

EXPORTAMOS LOS 3 DFs EN formato JSON.GZ

In [ ]:
#Exportamos los data frame
grabarJsonGz(df_spent_by_user,'spent_by_user')
grabarJsonGz(df_perc_recommend_by_user,'perc_recommend_by_user')
grabarJsonGz(df_count_items_by_user,'count_items_by_user')

.

.


.

2) **<u>def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): </u>**

    Cantidad de usuarios que realizaron reviews entre las fechas dadas y, 
    el porcentaje de recomendación de los mismos en base a reviews.recommend.

CREAREMOS 2 DFs REDUCIDOS (COMO SI FUESEN  VISTAS) para cada solicitud del ENDPOINT

df_user_reviews_posted

df_perc_recommend_by_user (creado para el Endo Point Anterior)

In [ ]:
df_user_reviews_posted = df_user_reviews[['user_id','item_id','posted','recommend']]
df_user_reviews_posted.head()

In [ ]:
df_user_reviews_posted.shape[0]

In [ ]:
#Exportamos los data frame
grabarJsonGz(df_user_reviews_posted,'user_reviews_posted')

.

.


.

3) **<u>def genre ( género : str ):</u>**  

    Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [ ]:
df_user_items[['item_id','playtime_forever']]

In [ ]:
# Contar la cantidad máxima de géneros por 'item_id'
max_genres_count = df_output_steam_json['genres'].str.count(',').max() + 1  # Agregar 1 para contar el primer género

print("La cantidad máxima de géneros por 'item_id' es:", max_genres_count)

In [ ]:
df_output_steam_json[df_output_steam_json['genres'] == '']

In [ ]:
# Dividir la columna 'genres' en géneros individuales y crear un conjunto de géneros únicos
unique_genres_set = set(df_output_steam_json['genres'].str.split(',').explode().str.strip())

# Convertir el conjunto de géneros únicos en una lista
unique_genres_list = list(unique_genres_set)

# Mostrar la lista de géneros únicos
print(unique_genres_list)
print(len(unique_genres_list))

In [ ]:
# Elimina filas con valores NaN en la columna 'item_id'
df_user_items_tmp = df_user_items.dropna(subset=['item_id'])

# Asegurarse de que la columna 'item_id' sea de tipo int64
df_user_items_tmp.loc[:, 'item_id'] = df_user_items_tmp['item_id'].astype(int)

# Paso 1: Dividir la columna 'genres' en géneros individuales
df_genres_split = df_output_steam_json['genres'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('genre').str.strip()

# Paso 2: Unir los DataFrames utilizando 'item_id'
df_merged = df_user_items_tmp.merge(df_genres_split, left_on='item_id', right_index=True)

# Paso 3: Calcular el ranking de géneros en función de 'playtime_forever'
genre_ranking = df_merged.groupby('genre')['playtime_forever'].sum().reset_index()

# Ordenar el DataFrame en función del tiempo total de juego en orden descendente
genre_ranking = genre_ranking.sort_values(by='playtime_forever', ascending=False)

#Reseteamos indices
genre_ranking.reset_index(drop=True, inplace=True)

# Mostrar el ranking de géneros
genre_ranking.head(10)

In [ ]:
genre_ranking.shape[0]

In [ ]:
genre_ranking['genre'].unique()

In [ ]:
#Creamos un DF vacio
df_genre_ranking = pd.DataFrame()

# Reiniciar el índice y agregar una nueva columna de ranking
df_genre_ranking = genre_ranking[['genre']].reset_index(drop=True)

# Renombrar la columna de ranking
df_genre_ranking.rename(columns={'index': 'ranking'}, inplace=True)


In [ ]:
#Exportamos los data frame
grabarJsonGz(df_genre_ranking,'genre_ranking')

.

.

.

4) **<u>def userforgenre ( género : str ):</u>**  

    Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [ ]:
# Dividir la columna genres en múltiples filas
df_genres_split = df_output_steam_json['genres'].str.split(',').explode().str.strip()

# Unir los DataFrames para obtener toda la información necesaria
df_user_genre_ranking = df_user_items.merge(df_genres_split.to_frame(), left_on='item_id', right_index=True)

#Seleccionamos columnas
df_user_genre_ranking = df_user_genre_ranking.drop(columns=['items_count', 'steam_id', 'item_name', 'playtime_2weeks'])
#Reordenamos las columnas
df_user_genre_ranking = df_user_genre_ranking[['genres', 'user_id', 'item_id', 'playtime_forever', 'user_url']]

#Visualizamos
df_user_genre_ranking.head()

In [ ]:
#Exportamos los data frame
grabarJsonGz(df_user_genre_ranking,'user_genre_ranking')

.

.

.

5) **<u>def developer( desarrollador : str ): </u>**  

    Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:

In [ ]:
# Crear un nuevo DataFrame con el nombre df_output_steam_tmp copiando df_output_steam_json
df_output_steam_tmp = df_output_steam_json.copy()

# Extraer el año de la columna 'release_date'
df_output_steam_tmp['year'] = df_output_steam_tmp['release_date'].str[:4]

# Convertir la columna 'year' a tipo numérico
df_output_steam_tmp['year'] = pd.to_numeric(df_output_steam_tmp['year'], errors='coerce')

# Filtrar los registros con año igual a 1900
df_output_steam_tmp = df_output_steam_tmp[df_output_steam_tmp['year'] != 1900]

# Calcular 'items_count' por desarrollador y año
developer_items_count = df_output_steam_tmp.groupby(['developer', 'year'])['item_id'].count().reset_index()
developer_items_count.rename(columns={'item_id': 'items_count'}, inplace=True)

# Calcular 'free_content' por desarrollador y año
developer_free_games = df_output_steam_tmp[df_output_steam_tmp['price'] == 0].groupby(['developer', 'year'])['item_id'].count().reset_index()
developer_free_games.rename(columns={'item_id': 'free_count'}, inplace=True)

# Combinar los DataFrames para calcular el porcentaje
df_content_developer = developer_items_count.merge(developer_free_games, on=['developer', 'year'], how='left')
df_content_developer['free_content'] = (df_content_developer['free_count'] / df_content_developer['items_count']) * 100
df_content_developer.drop(columns=['free_count'], inplace=True)

#Si no encuentra elementos con precio 0, ese registro va a quedar con Nan cuando deberia ser cero.
df_content_developer['free_content'].fillna(0, inplace=True)

# Visualizamos
df_content_developer.head()

In [ ]:
#Exportamos los data frame
grabarJsonGz(df_content_developer,'content_developer')

.

.

.

6) **<u>def sentiment_analysis( año : int ): </u>**  

    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [ ]:
# Crear el DataFrame df_review_year con la columna 'year'
df_reviews_year = df_output_steam_json[['year']].drop_duplicates().copy()

# Agrupa y suma los valores de sentiment_analysis en df_user_reviews por año
sentiment_year = df_user_reviews.groupby(['year', 'sentiment_analysis'])['sentiment_analysis'].count().unstack(fill_value=0).reset_index()

# Agregar las columnas 'Negativos', 'Neutros' y 'Positivos' en df_review_year
df_reviews_year = df_reviews_year.merge(sentiment_year, on='year', how='left')

# Cambiar los nombres de las columnas para mayor claridad
df_reviews_year.rename(columns={0: 'Negativos', 1: 'Neutros', 2: 'Positivos'}, inplace=True)

# Llenar NaN con 0 en las columnas 'Negativos', 'Neutros' y 'Positivos'
df_reviews_year.fillna(0, inplace=True)

# Convertir las columnas 'Negativos', 'Positivos' y 'Neutros' a enteros
df_reviews_year['Negativos'] = df_reviews_year['Negativos'].astype(int)
df_reviews_year['Positivos'] = df_reviews_year['Positivos'].astype(int)
df_reviews_year['Neutros']   = df_reviews_year['Neutros'].astype(int)

In [ ]:
df_reviews_year.shape

In [ ]:
df_reviews_year.head(20)

In [ ]:
df_reviews_year[(df_reviews_year['Negativos'] != 0) | (df_reviews_year['Positivos'] != 0) | (df_reviews_year['Neutros'] != 0)].head()

In [ ]:
#Años con reviews
df_reviews_year[(df_reviews_year['Negativos'] != 0) | (df_reviews_year['Positivos'] != 0) | (df_reviews_year['Neutros'] != 0)]['year'].unique()

In [ ]:
#Exportamos los data frame
grabarJsonGz(df_reviews_year,'reviews_year')

.

.

.

# MACHINE LEARNING

.

.

# MODELO DE RECOMENDACION ITEM-ITEM

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import random

In [ ]:
df_output_steam_json.head()

In [ ]:
df_user_reviews.head()

In [ ]:
df_user_items.head()

.

.


# PREPARACION DE LOS DATOS

.


VAMOS A GENERAR UN ARCHIVO QUE SERA UTILIZADO COMO INPUT EN NUESTRO MODELO DE MACHINE LEARNING (RECOMENDACION)

.


In [ ]:
# Filtrar columnas relevantes de cada conjunto de datos para el modelo de RECOMENDACION
df_steam_recomendacion   = df_output_steam_json[['item_id', 'genres']]
df_reviews_recomendacion = df_user_reviews[['user_id', 'item_id', 'sentiment_analysis']]
#En este caso eliminamos los registros que teniamos sin item-id, a los cuales les imputamos -1
df_items_recomendacion = df_user_items[df_user_items['item_id'] != -1][['user_id', 'item_id']]

In [ ]:
# Fusionar los datos en un solo conjunto
merged_data = df_reviews_recomendacion.merge(df_items_recomendacion, on=['user_id', 'item_id'], how='inner')
merged_data = merged_data.merge(df_steam_recomendacion, on='item_id', how='left')

In [ ]:
merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
# Convertir géneros en listas
merged_data['genres'] = merged_data['genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
merged_data.head()

In [ ]:
# Contar cuántos registros tienen listas vacías en la columna 'genres'
registros_con_listas_vacias = merged_data[merged_data['genres'].apply(len) == 0]
cantidad_registros_con_listas_vacias = len(registros_con_listas_vacias)

print("Cantidad de registros con listas vacías en la columna 'genres':", cantidad_registros_con_listas_vacias)

In [ ]:
# Eliminar registros con listas vacías en la columna 'genres'
merged_data = merged_data[merged_data['genres'].apply(len) > 0]

In [ ]:
merged_data.shape

In [ ]:
merged_data.columns

.


# MATRIZ DE CARACTERISTICAS

.


In [ ]:
# Eliminar registros con 'genres' vacíos o NaN
merged_data = merged_data.dropna(subset=['genres'])
merged_data = merged_data[merged_data['genres'].astype(bool)]

# Calcular la moda de sentimiento (sentiment_analysis) para cada juego
mode_sentiment = merged_data.groupby('item_id')['sentiment_analysis'].agg(lambda x: statistics.mode(x) if len(x) > 0 else None).reset_index()
mode_sentiment.rename(columns={'sentiment_analysis': 'mode_sentiment'}, inplace=True)

# Utilizar la función explode para descomponer la lista de géneros en filas separadas
genres_expanded = merged_data.explode('genres')

# Codificar géneros como variables dummy (one-hot encoding)
genres_dummy = pd.get_dummies(genres_expanded['genres'], prefix='genre')

# Combinar géneros dummy por 'item_id' utilizando sum()
genres_dummy = genres_dummy.groupby(genres_expanded['item_id']).sum()

# Reiniciar el índice
genres_dummy.reset_index(inplace=True)

# Combinar moda y dummies 
features = mode_sentiment.merge(genres_dummy, on='item_id', how='left')

# Mostrar una muestra del conjunto de datos fusionado con la moda de sentimiento y codificación one-hot de géneros
print(features.head())

In [ ]:
features.head()

In [ ]:
features.shape

In [ ]:
features.columns

In [ ]:
# Llenar valores NaN con 0 en el conjunto de datos 'features'
features.fillna(0, inplace=True)

# Calcular la matriz de similitud del coseno
similarities = cosine_similarity(features.drop(columns=['item_id']))

# Crear un DataFrame con la matriz de similitud
df_similarities = pd.DataFrame(similarities, index=features['item_id'], columns=features['item_id'])

# Mostrar una muestra de la matriz de similitud
print(df_similarities.head())

In [ ]:
df_similarities.shape

LA TABLA ES UN POCO GRANDE, CON LO CUAL VAMOS A APLICAR ALGUNA TECNICA PARA REDUCIR EL ESPACIO.

SABEMOS QUE MI API SOLO RECOMENDARA 5 JUEGOS SIMILARES, ENTONCES NO ME INTERESA GUARDAR MAS DE 5 ITEMs SIMILARES POR CADA ITEM. ESTO REDUCIRIA CONSIDERABLEMENTE EL ESPACIO OCUPADA POR EL ARCHIVO/TABLA/DF. ADEMAS, LA RESPUESTA DE LA API SERIA MUCHO MAS RAPIDA.

In [ ]:
df_similarities.index = df_similarities.index.astype(int)
recommendations_dict = {}

for item_id, row in df_similarities.iterrows():
    item_id = int(item_id)
    similar_games = df_similarities.loc[item_id].sort_values(ascending=False)
    similar_games.index = similar_games.index.astype(int)
    top_similar_games = similar_games[similar_games.index != item_id].index[:5]        
    # Almacenar las recomendaciones en el diccionario
    recommendations_dict[item_id] = top_similar_games.tolist()


# Convertir el diccionario en un nuevo DataFrame
df_reduced_recommendations = pd.DataFrame.from_dict(recommendations_dict, orient='index')

# Asignar nombres a las columnas
df_reduced_recommendations.columns = ['recommend_1', 'recommend_2', 'recommend_3', 'recommend_4', 'recommend_5']

# Asegurarse de que el índice sea 'item_id'
df_reduced_recommendations.index.name = 'item_id'
df_reduced_recommendations.index = df_reduced_recommendations.index.astype(int)

df_reduced_recommendations

In [ ]:
# Mover el índice a una columna llamada 'item_id' y restablecer el índice
df_reduced_recommendations['item_id'] = df_reduced_recommendations.index
df_reduced_recommendations.reset_index(drop=True, inplace=True)

#Exportamos la matriz de caracteristicas para poder consumirla en la API
grabarJsonGz(df_reduced_recommendations,'df_recommendations')

GENERAREMOS UN ARCHIVO CON EL RANKING DE JUEGOS EN FUNCION A LA CANTIDAD DE REVIEWS POSITIVAS QUE SE DETECTARON EN EL ANALISIS DE SENTIMIENTO. DE ESA LISTA DEVOLVEREMOS SOLO EL TOP5.

UTILIZAREMOS ESTA LISTA PARA DEVOLVER EN CASO QUE EL ITEM ID INGRESADO EN LA API NO ESTE EN NUESTRO ARCHIVO DE RECOMENDACIONES.

In [ ]:
# Filtrar las filas donde sentiment_analysis es igual a 2
positive_reviews = df_user_reviews[df_user_reviews['sentiment_analysis'] == 2]

# Calcular el recuento de reviews positivas por item_id
positive_reviews_count = positive_reviews.groupby('item_id')['sentiment_analysis'].count().reset_index()

# Renombrar la columna de recuento
positive_reviews_count.columns = ['item_id', 'positive_reviews_count']

# Ordenar el DataFrame por la columna 'positive_reviews_count' en orden descendente y tomar los primeros 5 registros
top_5_games = positive_reviews_count.sort_values(by='positive_reviews_count', ascending=False).head(5)

# Seleccionar solo la columna 'item_id'
top_5_item_ids = top_5_games['item_id']

# Crear un nuevo DataFrame solo con la columna 'item_id'
df_default_recommend_5 = pd.DataFrame({'item_id': top_5_item_ids})

#Casteamos el item_id por las dudas
df_default_recommend_5['item_id'] = df_default_recommend_5['item_id'].astype(int)

In [ ]:
#Exportamos
grabarJsonGz(df_default_recommend_5,'default_recommend_5')